#Importación de Librerias 

Son las librerias necesarias para la ejecución del codigo, en donde se pueden obtener los datos de Yahoo! Finanzas de una lista de emisoras.


In [ ]:
#Ejecutar pip en la primera ejecución
#!pip install yahoo-fin
#!pip install requests_html

from yahoo_fin import stock_info
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Carga variables y empresas 

Lista de las emisoras a extraer información, así como las fechas iniciales y finales de la información requerida

In [ ]:

#Variables iniciales
fecha_inicio='2021-01-01'
fecha_final="2021-09-09"
precio_limite=100

#Carga empresas a evaluar
emp= pd.read_csv('/content/drive/MyDrive/Escrito/Codigo/Portafolio/Emisoras.csv',encoding='latin-1')   #Actualizar Ruta del archivo
emp.head()

#Definición de emisoras 
lista_emp= np.array(emp['Emisora'])


#Extracción de datos

Funciones para extraer los precios de cierre de una emisora (precio_cierre) o una lista de emisoras (precios) todo esto a través de la Api yahoo_fin.

También se puede extrar el volumen de transacciones de una emisora (volumen) o grupo de emisoras (volums)

In [ ]:
#Función para obtener precios de cierre
def precio_cierre(emisora,fe_ini,fe_fin):
  stock_info.get_quote_table(emisora)
  base= pd.DataFrame(stock_info.get_data(emisora,fe_ini,fe_fin))
  base=base.drop(labels=['adjclose','open','high','volume','low','ticker'],axis=1,inplace=False)
  base.columns= [emisora]
  return base

#Datos de los precios de cierre de las acciones
def precios(lista_em,fecha_inicio,fecha_final):
  base=pd.DataFrame()
  #print(base)
  for le in lista_em:
    precio = precio_cierre(le,fecha_inicio,fecha_final)
    #precio.get('timestamp')
    #print(precio)
    if len(base)!=0:
      #base.get('timestamp')
      base= pd.concat([base,precio[le]],axis=1,)
    else:
       base=precio
    #print(base)
  return base.fillna(0)

In [ ]:
#Función para obtener volumenes de transacciones
def volumen(emisora,fe_ini,fe_fin):
  stock_info.get_quote_table(emisora)
  base= pd.DataFrame(stock_info.get_data(emisora,fe_ini,fe_fin))
  base=base.drop(labels=['close','adjclose','open','high','low','ticker'],axis=1,inplace=False)
  base.columns= [emisora]
  return base

#Datos de los volumnes de las acciones
def volums(lista_em,fecha_inicio,fecha_final):
  base=pd.DataFrame()
  for le in lista_em:
    volum = volumen(le,fecha_inicio,fecha_final)
    if len(base)!=0:
      base= pd.concat([base,volum[le]],axis=1,)
    else:
       base=volum
  return base.fillna(0)

#Extracción de Datos para Drive

En el caso que la información de los precios es almacenada en un csv en drive, se puede extraer la información a través de las funciones de abajo

In [ ]:
#Extracción de información desde Yahoo! Finanzas
def preciost(lista_em,fecha_inicio,fecha_final):
  base= pd.DataFrame(stock_info.get_data(lista_em[0],fecha_inicio,fecha_final))
  for le in lista_em[1:]:
    precio = pd.DataFrame(stock_info.get_data(le,fecha_inicio,fecha_final))
    base= pd.concat([base,precio])#,axis=1,)
  return base.fillna(0)

#Carga de la información desde drive
def cargadatos(path):
  base=pd.read_csv(path,index_col='Date',)
  base.index = pd.to_datetime(base.index)
  return base

#Datos de los precios de cierre de las acciones
def precios1 (path,lista_em,fecha_inicio,fecha_final):
  datos=cargadatos(path)
  base=pd.DataFrame()
  for le in lista_em:
    precio=pd.DataFrame(datos[((datos.index >= fecha_inicio) & (datos.index<=fecha_final)) & (datos.ticker==le)]['close'])
    precio.columns= [le]
    if len(base)!=0:
      base= pd.concat([base,precio[le]],axis=1,)
    else:
       base=precio
  return base.fillna(0)

#Rendimiento y Volatilidad

Calculo de metricas necesarias para la creación y optimización de portafolios de inversión, estas métricas son desde el rendimiento, rendimiento medio y riesgo de cada una de las emisoras.

In [ ]:
#Calculo de Rendimientos 
def rendimientos(precios):
  rend=(precios/precios.shift(1))-1
  return rend.replace([np.inf,-np.inf],np.nan).dropna() 

#Rendimiento promedio y Riesgo
def metricasRV(rendimiento):
  metrica=pd.DataFrame({'Media':rendimiento.mean(),'Riesgo':rendimiento.std()},index=rendimiento.columns)
  metrica.sort_values(by=['Media'],ascending=False)
  return metrica

#Filtro de emisoras

Filtro de emisoras a traves del máximo rendimiento promedio obtenido de un conjunto de rendimientos

In [ ]:
def top(base,n):
  met=pd.DataFrame({'Media':base.mean(),'Riesgo':base.std()},index=base.columns)
  met=met.sort_values(by=['Media'],ascending=False)
  return met.index[:n]

#Métricas de Trading 

Funciones que calculan dos técnicas de Trading Financiero:
Media Moviles: promedio de los precios en un determinado periodo de tamaño n
RSI: Indices que permite identificar la fuerza de la tendencia de los precios de una emisora, a traves de periodos de tamaño n

In [ ]:
#Media Móvil
def media_movil(base,periodo,emisora):
  return base.rolling(periodo).mean()
  #col='MM'+str(periodo)+'_'+emisora.replace('.MX','')
  #base[col]= base.rolling(periodo).mean()
  #return base[col]

#RSI
def rsi(base,periodo,emisora):
  base['dif']=base.diff()
  base['pos']=base['dif'].clip(lower=0)
  base['neg']=base['dif'].clip(upper=0)*-1
  base['RS']=base['pos'].rolling(periodo).mean()/base['neg'].rolling(periodo).mean()
  col= 'RSI'+'_'+emisora.replace('.MX','')
  base[col]=1-(1/(1+base['RS']))
  return base[col]

#Datos para Machine Learning

Funciones para extracción de información para el entrenamiento del modelo de Machine Learning seleccionado, en este caso es una Red Neuronal Convolucional.

In [ ]:
#Función para obtener los datos de cierre de cada acción
def datos_cierre(emisora,fe_ini,fe_fin,filtro):
  lista = ['close','adjclose','open','high','volume','low','ticker']
  lista.remove(filtro)
  stock_info.get_quote_table(emisora)
  base= pd.DataFrame(stock_info.get_data(emisora,fe_ini,fe_fin))
  base=base.drop(labels=lista,axis=1,inplace=False)
  col=filtro[:3]+'_'+emisora.replace('.MX','')
  base.columns= [col]
  return base

#Extracción de los datos de cierre de las acciones
def ext_datos_ML(lista_em,fecha_inicio,fecha_final):
  filtros =['close','volume','open','low','high']
  base=pd.DataFrame()
  for fil in filtros: 
    for le in lista_em:
      precio = datos_cierre(le,fecha_inicio,fecha_final,fil)
      if len(base)!=0:
        base= pd.concat([base,precio[precio.columns]],axis=1,)
      else:
        base=precio
      if fil=='close':
        periodos=[10,30,60]
        for p in periodos:
          MM='MM'+str(p)+'_'+le.replace('.MX','')
          MM_= media_movil(precio,p,le)
          MM_.columns=[MM]
          base= pd.concat([base,MM_[MM]],axis=1,)
        base['RSI'+str(14)+'_'+le.replace('.MX','')]=rsi(precio,14,le)
  return base.dropna()#fillna(0)

#Valores del portafolio optimo
def datos_port(lista_em,pesos,fecha_inicio,fecha_final):
  filtros =['close','volume','open','low','high']
  base=pd.DataFrame()
  for fil in filtros:
    indice=0 
    for le in lista_em:
      precio = datos_cierre(le,fecha_inicio,fecha_final,fil)
      precio.columns=[fil]
      precio.get('timestamp')
      if indice==0:
        valor=precio[fil]*pesos[indice]
        valor.columns=[fil]
        indice+=1
      else:
        valor+=precio[fil]*pesos[indice]
    if len(base)!=0:
      base.get('timestamp')
      base=base.join(valor)
    else:
      base=valor
    print('a')
    if fil=='close':
      periodos=[10,30,60]
      col=[str(fil)]
      for p in periodos:
        MM='MM'+str(p)+'_Port'
        col.append(str(MM))
        MM_= media_movil(valor,p,le)
        base= pd.concat([base,MM_],axis=1,)
      base.columns=col
      base['RSI'+str(14)+'_Port']=rsi(valor,14,le)
      base['Rend']=(base['close']/base['close'].shift(1))-1
      base['clase']= np.where(base['close'].diff()>0,1,0)
      base['clase2']= np.where(base['clase'].shift(-1)>0,1,0)
  return base.dropna()

#Valores de evaluación del portafolio optimo
def datos_eval(lista_em,pesos,fecha_inicio,fecha_final):
  filtros =['close']#,'volume','open','low','high']
  base=pd.DataFrame()
  for fil in filtros:
    indice=0 
    for le in lista_em:
      precio = datos_cierre(le,fecha_inicio,fecha_final,fil)
      precio.columns=[fil]
      if indice==0:
        valor=precio[fil]*pesos[indice]
        valor.columns=[fil]
        indice+=1
      else:
        valor+=precio[fil]*pesos[indice]
    if len(base)!=0:
      base=base.join(valor)
    else:
      base=valor
    if fil=='close':
      periodos=[10,30,60]
      col=[str(fil)]
      for p in periodos:
        MM='MM'+str(p)+'_Port'
        col.append(str(MM))
        MM_= media_movil(valor,p,le)
        base= pd.concat([base,MM_],axis=1,)
      base.columns=col
      base['RSI'+str(14)+'_Port']=rsi(valor,14,le)
      base['Rend']=(base['close']/base['close'].shift(1))-1
  return base.dropna()

#Datos fuente para modelo de Machine Learning
def datos_ML(datos,tiempo=3):
  columnas=list(datos.columns)
  columnas.remove('clase')
  for c in columnas:
    for i in range(tiempo):
      col=f'{c}_tmp{i+1}'
      datos[col]= datos[c].shift(i+1)
  return datos.dropna()

  #Datos fuente para modelo de Machine Learning
def datos_ML1(datos,tiempo=3):
  for t in range(tiempo+1):
    col=f'cierre_lag{t+1}'
    datos[col]=datos['close'].shift(t+1)
  datos['max']=datos['close'].rolling(5).max()
  datos['min']=datos['close'].rolling(5).min()
  datos['prom_rend']=datos['Rend'].rolling(5).mean()
  datos['prom_ries']=datos['Rend'].rolling(5).std()
  return datos.dropna()


  #Datos fuente para modelo de Machine Learning
def datos_ML2(datos,tiempo=3):
  columnas=list(datos.columns)
  columnas.remove('clase')
  columnas.remove('clase2')
  for c in columnas:
    for i in range(tiempo):
      col=f'{c}_tmp{i+1}'
      datos[col]= datos[c].shift(i+1)
  #for t in range(tiempo+1):
  #  col=f'cierre_lag{t+1}'
  #  datos[col]=datos['close'].shift(t+1)
  #t=max(tiempo,)
  datos['max']=datos['close'].rolling(5).max()
  datos['min']=datos['close'].rolling(5).min()
  datos['prom_rend']=datos['Rend'].rolling(5).mean()
  datos['prom_ries']=datos['Rend'].rolling(5).std()
  return datos.dropna()


#Datos para Machine Learning (Drive)

Funciones para extracción de información para el entrenamiento del modelo de Machine Learning seleccionado, en este caso es una Red Neuronal Convolucional. Cuando la información se encuentra almacenada en una carpeta Drive o temporal.

In [ ]:
#Función para obtener los datos de cierre de cada acción
def datos_cierre1(path,emisora,fe_ini,fe_fin,filtro):
  datos= cargadatos(path)
  base=pd.DataFrame(datos[((datos.index >= fe_ini) & (datos.index<=fe_fin)) & (datos.ticker==emisora)][filtro])
  col=filtro[:3]+'_'+emisora.replace('.MX','')
  base.columns= [col]
  return base

#Extracción de los datos de cierre de las acciones
def ext_datos1_ML(lista_em,fecha_inicio,fecha_final):
  filtros =['close','volume','open','low','high']
  base=pd.DataFrame()
  for fil in filtros: 
    for le in lista_em:
      precio = datos_cierre1(le,fecha_inicio,fecha_final,fil)
      if len(base)!=0:
        base= pd.concat([base,precio[precio.columns]],axis=1,)
      else:
        base=precio
      if fil=='close':
        periodos=[10,30,60]
        for p in periodos:
          MM='MM'+str(p)+'_'+le.replace('.MX','')
          MM_= media_movil(precio,p,le)
          MM_.columns=[MM]
          base= pd.concat([base,MM_[MM]],axis=1,)
        base['RSI'+str(14)+'_'+le.replace('.MX','')]=rsi(precio,14,le)
  return base.dropna()

#Valores del portafolio optimo
def datos1_port(path,lista_em,pesos,fecha_inicio,fecha_final):
  filtros =['close','volume','open','low','high']
  base=pd.DataFrame()
  for fil in filtros:
    indice=0 
    for le in lista_em:
      precio = datos_cierre1(path,le,fecha_inicio,fecha_final,fil)
      precio.columns=[fil]
      precio.get('timestamp')
      if indice==0:
        valor=precio[fil]*pesos[indice]
        valor.columns=[fil]
        indice+=1
      else:
        valor+=precio[fil]*pesos[indice]
    if len(base)!=0:
      base.get('timestamp')
      base=base.join(valor)
    else:
      base=valor
    if fil=='close':
      periodos=[10,30,60]
      col=[str(fil)]
      for p in periodos:
        MM='MM'+str(p)+'_Port'
        col.append(str(MM))
        MM_= media_movil(valor,p,le)
        base= pd.concat([base,MM_],axis=1,)
      base.columns=col
      base['RSI'+str(14)+'_Port']=rsi(valor,14,le)
      base['Rend']=(base['close']/base['close'].shift(1))-1
      base['clase']= np.where(base['close'].diff()>0,1,0)
      base['clase2']= np.where(base['clase'].shift(-1)>0,1,0)
  return base.dropna()

#Valores de evaluación del portafolio optimo
def datos1_eval(path,lista_em,pesos,fecha_inicio,fecha_final):
  filtros =['close']#,'volume','open','low','high']
  base=pd.DataFrame()
  for fil in filtros:
    indice=0 
    for le in lista_em:
      precio = datos_cierre1(path,le,fecha_inicio,fecha_final,fil)
      precio.columns=[fil]
      if indice==0:
        valor=precio[fil]*pesos[indice]
        valor.columns=[fil]
        indice+=1
      else:
        valor+=precio[fil]*pesos[indice]
    if len(base)!=0:
      base=base.join(valor)
    else:
      base=valor
    if fil=='close':
      periodos=[10,30,60]
      col=[str(fil)]
      for p in periodos:
        MM='MM'+str(p)+'_Port'
        col.append(str(MM))
        MM_= media_movil(valor,p,le)
        base= pd.concat([base,MM_],axis=1,)
      base.columns=col
      base['RSI'+str(14)+'_Port']=rsi(valor,14,le)
      base['Rend']=(base['close']/base['close'].shift(1))-1
  return base.dropna()
